# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [ ]:
# Carregar os dados de treinamento e teste
X_train = pd.read_csv('X_train.txt', index_col=0)
y_train = pd.read_csv('y_train.txt', index_col=0)
X_test = pd.read_csv('X_test.txt', index_col=0)
y_test = pd.read_csv('y_test.txt', index_col=0)

# Dividir a base de treinamento em treino e validação (75% treino, 25% validação)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Verificar o tamanho dos conjuntos de dados resultantes
print("Tamanho do conjunto de treinamento:", X_train.shape)
print("Tamanho do conjunto de validação:", X_val.shape)
print("Tamanho do conjunto de teste:", X_test.shape)

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [ ]:
# Inicializar e treinar a árvore de decisão
clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)

# Obter a importância das variáveis
importances = clf.feature_importances_

# Criar um DataFrame com as importâncias das variáveis
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Selecionar as três variáveis mais importantes
top_3_variables = importance_df.head(3)['Feature'].tolist()

print("As três variáveis mais importantes são:", top_3_variables)

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Selecionar apenas as três melhores variáveis
X_train_top3 = X_train[top_3_variables]
X_val_top3 = X_val[top_3_variables]

# Inicializar uma lista para armazenar as árvores de decisão
trees = []

# Range de valores de ccp_alpha para avaliar
ccp_alphas = np.linspace(0, 0.1, 10)

# Treinar uma árvore de decisão para cada valor de ccp_alpha
for ccp_alpha in ccp_alphas:
    # Inicializar e treinar a árvore de decisão
    tree = DecisionTreeClassifier(max_depth=4, ccp_alpha=ccp_alpha, random_state=42)
    tree.fit(X_train_top3, y_train)
    trees.append(tree)

# Avaliar o desempenho de cada árvore na base de validação
validation_accuracies = []
for tree in trees:
    y_val_pred = tree.predict(X_val_top3)
    accuracy = accuracy_score(y_val, y_val_pred)
    validation_accuracies.append(accuracy)

# Encontrar o índice do melhor valor de ccp_alpha
best_alpha_index = np.argmax(validation_accuracies)
best_alpha = ccp_alphas[best_alpha_index]

print("Melhor ccp_alpha encontrado:", best_alpha)


### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Selecionar apenas as três melhores variáveis na base de testes
X_test_top3 = X_test[top_3_variables]

# Inicializar e treinar a árvore de decisão com o melhor ccp_alpha encontrado
best_tree = DecisionTreeClassifier(max_depth=4, ccp_alpha=best_alpha, random_state=42)
best_tree.fit(X_train_top3, y_train)

# Fazer previsões na base de testes
y_test_pred = best_tree.predict(X_test_top3)

# Avaliar o desempenho da árvore na base de testes
accuracy = accuracy_score(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)

print("Acurácia na base de testes:", accuracy)
print("Matriz de Confusão:")
print(conf_matrix)